In [ ]:
from fastai.vision.all import *
from os import path

In [ ]:
file_path = '../data/posters/'

poster_df = pd.read_csv("../data/MovieGenre.csv", encoding = "ISO-8859-1")
poster_df = poster_df.drop_duplicates(subset=['imdbId'], keep="last")
poster_df["filename"] = poster_df["imdbId"].astype(str) + ".jpg"
poster_df = poster_df[poster_df["filename"].apply(lambda t: path.exists(file_path + t))]
poster_df = poster_df[poster_df["Genre"].apply(lambda t: isinstance(t, str))]
poster_df.head()

In [ ]:
def get_x(r):
    return file_path + r['filename']

def get_y(r):
    return r['Genre'].split('|')

dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   get_x = get_x, 
                   get_y = get_y,
                   item_tfms = RandomResizedCrop(128, min_scale = 0.35),
                   batch_tfms=aug_transforms())

In [ ]:
dls = dblock.dataloaders(poster_df)
dls.show_batch(nrows=1, ncols=3)

In [ ]:
f1score_multi = F1ScoreMulti()
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, metrics=f1score_multi)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(7, 3e-2)

In [ ]:
learn.show_results()

In [ ]:
learn.export('poster_predictor_fastai.pkl')